In [1]:
# import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
import lightgbm as lgb

In [2]:
# Display Options of DataFrame
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
# Loading Swap Rate Dataset
data = pd.read_csv("../cleanedData/merged.csv")

In [4]:
data['CMS Tenor'] = data['CMS Tenor'].str.removesuffix('y')
data['CMS Tenor'] = data['CMS Tenor'].astype(int)

In [5]:
# Find lag 1 by shifting the amount of data in one date
delay = len(data.loc[data['Value Date'] == "2022-09-02"])

# Realized Swap Rate Prev
data['Swap Rate Lag 1 (Real)'] = data['Swap Rate'].shift(delay)

In [6]:
data

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,-100,-0.01000,-227907.09877,1.00000,2023-09-04,10.00000,1.96225,2,0.02916,0.01916,0.00420,0.03790,0.03370,6,5,0.01496,0.01874,False,0.01496,0.17753,NaN
1,2022-09-02,dummyTrade1,USD,-50,-0.00500,-222208.40097,1.00000,2023-09-04,10.00000,-3.81234,2,0.02916,0.02416,0.00420,0.03790,0.03370,6,5,0.01996,0.01374,False,0.01374,0.17753,NaN
2,2022-09-02,dummyTrade1,USD,-25,-0.00250,-218960.92800,1.00000,2023-09-04,10.00000,4.47101,2,0.02916,0.02666,0.00420,0.03790,0.03370,6,5,0.02246,0.01124,False,0.01124,0.17753,NaN
3,2022-09-02,dummyTrade1,USD,-10,-0.00100,-216872.43011,1.00000,2023-09-04,10.00000,4.33340,2,0.02916,0.02816,0.00420,0.03790,0.03370,6,5,0.02396,0.00974,False,0.00974,0.17753,NaN
4,2022-09-02,dummyTrade1,USD,-5,-0.00050,-216146.31033,1.00000,2023-09-04,10.00000,5.67969,2,0.02916,0.02866,0.00420,0.03790,0.03370,6,5,0.02446,0.00924,False,0.00924,0.17753,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3919099,2024-01-12,dummyTrade9,USD,10,0.00100,-547852.05593,15.00000,2039-01-12,8.00000,5.00795,5,0.03039,0.03139,0.00440,0.03970,0.03530,6,10,0.02699,0.00831,False,0.00831,0.25391,0.03069
3919100,2024-01-12,dummyTrade9,USD,25,0.00250,-521228.58762,15.00000,2039-01-12,8.00000,5.13151,5,0.03039,0.03289,0.00440,0.03970,0.03530,6,10,0.02849,0.00681,False,0.00681,0.25391,0.03069
3919101,2024-01-12,dummyTrade9,USD,50,0.00500,-464905.98210,15.00000,2039-01-12,8.00000,3.34862,5,0.03039,0.03539,0.00440,0.03970,0.03530,6,10,0.03099,0.00431,False,0.00431,0.25391,0.03069
3919102,2024-01-12,dummyTrade9,USD,75,0.00750,-391315.04193,15.00000,2039-01-12,8.00000,1.06363,5,0.03039,0.03789,0.00440,0.03970,0.03530,6,10,0.03349,0.00181,False,0.00181,0.25391,0.03069


In [7]:
zero_subset = data['Zero Rate Shock'] == 0
shock_subset = -zero_subset
zero = data[zero_subset].reset_index(drop = True)
shock = data[shock_subset].reset_index(drop = True)

In [8]:
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real)
0,2022-09-02,dummyTrade1,USD,0,0.00000,-215402.65615,1.00000,2023-09-04,10.00000,5.01627,2,0.02916,0.02916,0.00420,0.03790,0.03370,6,5,0.02496,0.00874,False,0.00874,0.17753,NaN
1,2022-09-02,dummyTrade10,USD,0,0.00000,-616299.36924,1.00000,2023-09-04,10.00000,-59.82041,5,0.02725,0.02725,0.00440,0.03970,0.03530,4,10,0.02285,0.01245,False,0.01245,0.17753,NaN
2,2022-09-02,dummyTrade11,USD,0,0.00000,-608678.01098,1.00000,2023-09-04,10.00000,7.76260,10,0.02738,0.02738,0.00460,0.04170,0.03710,4,10,0.02278,0.01432,False,0.01432,0.17753,NaN
3,2022-09-02,dummyTrade12,USD,0,0.00000,-609360.66241,1.00000,2023-09-04,10.00000,25.32978,10,0.02738,0.02738,0.00460,0.04170,0.03710,3,10,0.02278,0.01432,False,0.01432,0.17753,NaN
4,2022-09-02,dummyTrade2,USD,0,0.00000,-215836.78721,1.00000,2023-09-04,10.00000,5.40938,2,0.02916,0.02916,0.00420,0.03790,0.03370,4,5,0.02496,0.00874,False,0.00874,0.17753,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.00000,-200752.70782,15.00000,2039-01-12,8.00000,-0.27797,10,0.03171,0.03171,0.00460,0.04170,0.03710,4,5,0.02711,0.00999,False,0.00999,0.25391,0.03188
326588,2024-01-12,dummyTrade6,USD,0,0.00000,-200966.21477,15.00000,2039-01-12,8.00000,-0.29635,10,0.03171,0.03171,0.00460,0.04170,0.03710,3,5,0.02711,0.00999,False,0.00999,0.25391,0.03188
326589,2024-01-12,dummyTrade7,USD,0,0.00000,-507396.66328,15.00000,2039-01-12,8.00000,5.05610,2,0.02983,0.02983,0.00420,0.03790,0.03370,6,10,0.02563,0.00807,False,0.00807,0.25391,0.03052
326590,2024-01-12,dummyTrade8,USD,0,0.00000,-508418.03933,15.00000,2039-01-12,8.00000,4.91148,2,0.02983,0.02983,0.00420,0.03790,0.03370,4,10,0.02563,0.00807,False,0.00807,0.25391,0.03052


In [9]:
delay = len(zero.loc[zero['Value Date'] == "2022-09-02"])
zero['Vols Prev'] = zero['Vols'].shift(delay)
zero['Vols Change'] = zero['Vols'] - zero['Vols Prev']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change
0,2022-09-02,dummyTrade1,USD,0,0.00000,-215402.65615,1.00000,2023-09-04,10.00000,5.01627,2,0.02916,0.02916,0.00420,0.03790,0.03370,6,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.00000,-616299.36924,1.00000,2023-09-04,10.00000,-59.82041,5,0.02725,0.02725,0.00440,0.03970,0.03530,4,10,0.02285,0.01245,False,0.01245,0.17753,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.00000,-608678.01098,1.00000,2023-09-04,10.00000,7.76260,10,0.02738,0.02738,0.00460,0.04170,0.03710,4,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.00000,-609360.66241,1.00000,2023-09-04,10.00000,25.32978,10,0.02738,0.02738,0.00460,0.04170,0.03710,3,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.00000,-215836.78721,1.00000,2023-09-04,10.00000,5.40938,2,0.02916,0.02916,0.00420,0.03790,0.03370,4,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.00000,-200752.70782,15.00000,2039-01-12,8.00000,-0.27797,10,0.03171,0.03171,0.00460,0.04170,0.03710,4,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264
326588,2024-01-12,dummyTrade6,USD,0,0.00000,-200966.21477,15.00000,2039-01-12,8.00000,-0.29635,10,0.03171,0.03171,0.00460,0.04170,0.03710,3,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264
326589,2024-01-12,dummyTrade7,USD,0,0.00000,-507396.66328,15.00000,2039-01-12,8.00000,5.05610,2,0.02983,0.02983,0.00420,0.03790,0.03370,6,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264
326590,2024-01-12,dummyTrade8,USD,0,0.00000,-508418.03933,15.00000,2039-01-12,8.00000,4.91148,2,0.02983,0.02983,0.00420,0.03790,0.03370,4,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264


In [10]:
# Find Risk of RAN notes contributed by swaption IV changes

# change in TV caused by vega (t=0) = previous vega (t-1)* vol change
zero['Vega Lag 1']  = zero['Vega'].shift(delay)
zero['Vega Risk Exposure'] = zero['Vols Change'] * zero['Vega Lag 1']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure
0,2022-09-02,dummyTrade1,USD,0,0.00000,-215402.65615,1.00000,2023-09-04,10.00000,5.01627,2,0.02916,0.02916,0.00420,0.03790,0.03370,6,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.00000,-616299.36924,1.00000,2023-09-04,10.00000,-59.82041,5,0.02725,0.02725,0.00440,0.03970,0.03530,4,10,0.02285,0.01245,False,0.01245,0.17753,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.00000,-608678.01098,1.00000,2023-09-04,10.00000,7.76260,10,0.02738,0.02738,0.00460,0.04170,0.03710,4,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.00000,-609360.66241,1.00000,2023-09-04,10.00000,25.32978,10,0.02738,0.02738,0.00460,0.04170,0.03710,3,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.00000,-215836.78721,1.00000,2023-09-04,10.00000,5.40938,2,0.02916,0.02916,0.00420,0.03790,0.03370,4,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.00000,-200752.70782,15.00000,2039-01-12,8.00000,-0.27797,10,0.03171,0.03171,0.00460,0.04170,0.03710,4,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.15633,-0.00041
326588,2024-01-12,dummyTrade6,USD,0,0.00000,-200966.21477,15.00000,2039-01-12,8.00000,-0.29635,10,0.03171,0.03171,0.00460,0.04170,0.03710,3,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.20701,-0.00055
326589,2024-01-12,dummyTrade7,USD,0,0.00000,-507396.66328,15.00000,2039-01-12,8.00000,5.05610,2,0.02983,0.02983,0.00420,0.03790,0.03370,6,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,5.04961,0.01331
326590,2024-01-12,dummyTrade8,USD,0,0.00000,-508418.03933,15.00000,2039-01-12,8.00000,4.91148,2,0.02983,0.02983,0.00420,0.03790,0.03370,4,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,4.26339,0.01124


In [11]:
zero['Vega Risk Exposure'].describe()

count   325620.00000
mean         0.02697
std         22.07968
min      -3300.85208
25%         -0.00761
50%          0.00000
75%          0.00829
max       5232.58544
Name: Vega Risk Exposure, dtype: float64

In [12]:
zero['Swap Rate Change'] = zero['Swap Rate'] - zero['Swap Rate Lag 1 (Real)']
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change
0,2022-09-02,dummyTrade1,USD,0,0.00000,-215402.65615,1.00000,2023-09-04,10.00000,5.01627,2,0.02916,0.02916,0.00420,0.03790,0.03370,6,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.00000,-616299.36924,1.00000,2023-09-04,10.00000,-59.82041,5,0.02725,0.02725,0.00440,0.03970,0.03530,4,10,0.02285,0.01245,False,0.01245,0.17753,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.00000,-608678.01098,1.00000,2023-09-04,10.00000,7.76260,10,0.02738,0.02738,0.00460,0.04170,0.03710,4,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.00000,-609360.66241,1.00000,2023-09-04,10.00000,25.32978,10,0.02738,0.02738,0.00460,0.04170,0.03710,3,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.00000,-215836.78721,1.00000,2023-09-04,10.00000,5.40938,2,0.02916,0.02916,0.00420,0.03790,0.03370,4,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.00000,-200752.70782,15.00000,2039-01-12,8.00000,-0.27797,10,0.03171,0.03171,0.00460,0.04170,0.03710,4,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.15633,-0.00041,-0.00016
326588,2024-01-12,dummyTrade6,USD,0,0.00000,-200966.21477,15.00000,2039-01-12,8.00000,-0.29635,10,0.03171,0.03171,0.00460,0.04170,0.03710,3,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.20701,-0.00055,-0.00016
326589,2024-01-12,dummyTrade7,USD,0,0.00000,-507396.66328,15.00000,2039-01-12,8.00000,5.05610,2,0.02983,0.02983,0.00420,0.03790,0.03370,6,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,5.04961,0.01331,-0.00069
326590,2024-01-12,dummyTrade8,USD,0,0.00000,-508418.03933,15.00000,2039-01-12,8.00000,4.91148,2,0.02983,0.02983,0.00420,0.03790,0.03370,4,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,4.26339,0.01124,-0.00069


In [13]:
zero['TV Lag 1'] = zero['TV'].shift(delay)
zero['TV Change'] = zero['TV'] - zero['TV Lag 1'] 
zero

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
0,2022-09-02,dummyTrade1,USD,0,0.00000,-215402.65615,1.00000,2023-09-04,10.00000,5.01627,2,0.02916,0.02916,0.00420,0.03790,0.03370,6,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-09-02,dummyTrade10,USD,0,0.00000,-616299.36924,1.00000,2023-09-04,10.00000,-59.82041,5,0.02725,0.02725,0.00440,0.03970,0.03530,4,10,0.02285,0.01245,False,0.01245,0.17753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-09-02,dummyTrade11,USD,0,0.00000,-608678.01098,1.00000,2023-09-04,10.00000,7.76260,10,0.02738,0.02738,0.00460,0.04170,0.03710,4,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-09-02,dummyTrade12,USD,0,0.00000,-609360.66241,1.00000,2023-09-04,10.00000,25.32978,10,0.02738,0.02738,0.00460,0.04170,0.03710,3,10,0.02278,0.01432,False,0.01432,0.17753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-09-02,dummyTrade2,USD,0,0.00000,-215836.78721,1.00000,2023-09-04,10.00000,5.40938,2,0.02916,0.02916,0.00420,0.03790,0.03370,4,5,0.02496,0.00874,False,0.00874,0.17753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326587,2024-01-12,dummyTrade5,USD,0,0.00000,-200752.70782,15.00000,2039-01-12,8.00000,-0.27797,10,0.03171,0.03171,0.00460,0.04170,0.03710,4,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.15633,-0.00041,-0.00016,-201031.68108,278.97326
326588,2024-01-12,dummyTrade6,USD,0,0.00000,-200966.21477,15.00000,2039-01-12,8.00000,-0.29635,10,0.03171,0.03171,0.00460,0.04170,0.03710,3,5,0.02711,0.00999,False,0.00999,0.25391,0.03188,0.25127,0.00264,-0.20701,-0.00055,-0.00016,-201241.28870,275.07393
326589,2024-01-12,dummyTrade7,USD,0,0.00000,-507396.66328,15.00000,2039-01-12,8.00000,5.05610,2,0.02983,0.02983,0.00420,0.03790,0.03370,6,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,5.04961,0.01331,-0.00069,-509747.56833,2350.90505
326590,2024-01-12,dummyTrade8,USD,0,0.00000,-508418.03933,15.00000,2039-01-12,8.00000,4.91148,2,0.02983,0.02983,0.00420,0.03790,0.03370,4,10,0.02563,0.00807,False,0.00807,0.25391,0.03052,0.25127,0.00264,4.26339,0.01124,-0.00069,-510774.02962,2355.99029


In [14]:
#zero['TV Change'].describe()

In [15]:
#zero[zero['TV Change'] == zero['TV Change'].max()]
#zero[(zero['Value Date'] == '2023-04-24') & (zero['Trade Name'] == "dummyTrade12")]

In [16]:
#Zero Shock data subset preprocessing Completed  

#Shock data subset Follows->>>

### MUST READ: For Shock data subset, use in place comparison with t=0 and baseline = zero rate, don't use other method like time comparison!!



In [17]:
# Use Joins (pd.merge) to map zero rate data for each of them
select_bucket = ['Value Date', 'Trade Name', 'TV', 'Expiry Bucket', 'Tenor Bucket', 'Vega']
zero_tojoin = zero[select_bucket]
zero_tojoin = zero_tojoin.rename(columns={'TV': 'TV Lag 1', 'Vega':'Vega Lag 1'}).reset_index(drop = True)

join_bucket = ['Value Date', 'Trade Name', 'Expiry Bucket', 'Tenor Bucket']
shock = shock.merge(zero_tojoin, on = join_bucket)
shock

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),TV Lag 1,Vega Lag 1
0,2022-09-02,dummyTrade1,USD,-100,-0.01000,-227907.09877,1.00000,2023-09-04,10.00000,1.96225,2,0.02916,0.01916,0.00420,0.03790,0.03370,6,5,0.01496,0.01874,False,0.01496,0.17753,NaN,-215402.65615,5.01627
1,2022-09-02,dummyTrade1,USD,-50,-0.00500,-222208.40097,1.00000,2023-09-04,10.00000,-3.81234,2,0.02916,0.02416,0.00420,0.03790,0.03370,6,5,0.01996,0.01374,False,0.01374,0.17753,NaN,-215402.65615,5.01627
2,2022-09-02,dummyTrade1,USD,-25,-0.00250,-218960.92800,1.00000,2023-09-04,10.00000,4.47101,2,0.02916,0.02666,0.00420,0.03790,0.03370,6,5,0.02246,0.01124,False,0.01124,0.17753,NaN,-215402.65615,5.01627
3,2022-09-02,dummyTrade1,USD,-10,-0.00100,-216872.43011,1.00000,2023-09-04,10.00000,4.33340,2,0.02916,0.02816,0.00420,0.03790,0.03370,6,5,0.02396,0.00974,False,0.00974,0.17753,NaN,-215402.65615,5.01627
4,2022-09-02,dummyTrade1,USD,-5,-0.00050,-216146.31033,1.00000,2023-09-04,10.00000,5.67969,2,0.02916,0.02866,0.00420,0.03790,0.03370,6,5,0.02446,0.00924,False,0.00924,0.17753,NaN,-215402.65615,5.01627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592507,2024-01-12,dummyTrade9,USD,10,0.00100,-547852.05593,15.00000,2039-01-12,8.00000,5.00795,5,0.03039,0.03139,0.00440,0.03970,0.03530,6,10,0.02699,0.00831,False,0.00831,0.25391,0.03069,-563257.81996,5.09762
3592508,2024-01-12,dummyTrade9,USD,25,0.00250,-521228.58762,15.00000,2039-01-12,8.00000,5.13151,5,0.03039,0.03289,0.00440,0.03970,0.03530,6,10,0.02849,0.00681,False,0.00681,0.25391,0.03069,-563257.81996,5.09762
3592509,2024-01-12,dummyTrade9,USD,50,0.00500,-464905.98210,15.00000,2039-01-12,8.00000,3.34862,5,0.03039,0.03539,0.00440,0.03970,0.03530,6,10,0.03099,0.00431,False,0.00431,0.25391,0.03069,-563257.81996,5.09762
3592510,2024-01-12,dummyTrade9,USD,75,0.00750,-391315.04193,15.00000,2039-01-12,8.00000,1.06363,5,0.03039,0.03789,0.00440,0.03970,0.03530,6,10,0.03349,0.00181,False,0.00181,0.25391,0.03069,-563257.81996,5.09762


In [18]:
# Finding TV change
shock['TV Change'] = shock['TV'] - shock['TV Lag 1']

# Vols here is 0 shock vols, rename to better represent
# Swap Rate here is swap rate at 0 shock
shock = shock.rename(columns={"Vols":"Vols Lag 1", "Swap Rate": "Swap Rate Lag 1"})

# Rename current swap rate to current shocked swap rate
shock = shock.rename(columns={"Swap Rate (Shocked)": "Swap Rate"})

In [19]:
# Renaming and dropping unused column, unifying naming of shock and zero data subset
shock['Swap Rate Change'] = shock['Zero Rate Shock (DEC)']
shock = shock.drop(columns= ['Zero Rate Shock (DEC)', 'Swap Rate Lag 1 (Real)'])

In [20]:
shock

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate Lag 1,Swap Rate,lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols Lag 1,TV Lag 1,Vega Lag 1,TV Change,Swap Rate Change
0,2022-09-02,dummyTrade1,USD,-100,-227907.09877,1.00000,2023-09-04,10.00000,1.96225,2,0.02916,0.01916,0.00420,0.03790,0.03370,6,5,0.01496,0.01874,False,0.01496,0.17753,-215402.65615,5.01627,-12504.44262,-0.01000
1,2022-09-02,dummyTrade1,USD,-50,-222208.40097,1.00000,2023-09-04,10.00000,-3.81234,2,0.02916,0.02416,0.00420,0.03790,0.03370,6,5,0.01996,0.01374,False,0.01374,0.17753,-215402.65615,5.01627,-6805.74481,-0.00500
2,2022-09-02,dummyTrade1,USD,-25,-218960.92800,1.00000,2023-09-04,10.00000,4.47101,2,0.02916,0.02666,0.00420,0.03790,0.03370,6,5,0.02246,0.01124,False,0.01124,0.17753,-215402.65615,5.01627,-3558.27184,-0.00250
3,2022-09-02,dummyTrade1,USD,-10,-216872.43011,1.00000,2023-09-04,10.00000,4.33340,2,0.02916,0.02816,0.00420,0.03790,0.03370,6,5,0.02396,0.00974,False,0.00974,0.17753,-215402.65615,5.01627,-1469.77395,-0.00100
4,2022-09-02,dummyTrade1,USD,-5,-216146.31033,1.00000,2023-09-04,10.00000,5.67969,2,0.02916,0.02866,0.00420,0.03790,0.03370,6,5,0.02446,0.00924,False,0.00924,0.17753,-215402.65615,5.01627,-743.65418,-0.00050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3592507,2024-01-12,dummyTrade9,USD,10,-547852.05593,15.00000,2039-01-12,8.00000,5.00795,5,0.03039,0.03139,0.00440,0.03970,0.03530,6,10,0.02699,0.00831,False,0.00831,0.25391,-563257.81996,5.09762,15405.76403,0.00100
3592508,2024-01-12,dummyTrade9,USD,25,-521228.58762,15.00000,2039-01-12,8.00000,5.13151,5,0.03039,0.03289,0.00440,0.03970,0.03530,6,10,0.02849,0.00681,False,0.00681,0.25391,-563257.81996,5.09762,42029.23234,0.00250
3592509,2024-01-12,dummyTrade9,USD,50,-464905.98210,15.00000,2039-01-12,8.00000,3.34862,5,0.03039,0.03539,0.00440,0.03970,0.03530,6,10,0.03099,0.00431,False,0.00431,0.25391,-563257.81996,5.09762,98351.83787,0.00500
3592510,2024-01-12,dummyTrade9,USD,75,-391315.04193,15.00000,2039-01-12,8.00000,1.06363,5,0.03039,0.03789,0.00440,0.03970,0.03530,6,10,0.03349,0.00181,False,0.00181,0.25391,-563257.81996,5.09762,171942.77803,0.00750


In [21]:
# TODO: Vols (SABR, pending)		Vols Change(SABR, pending)	 
# Vega Risk Exposure(SABR, pending)	

In [22]:
shock['TV Change'].describe() 

count   3592512.00000
mean      14591.99638
std       57968.50351
min     -208656.31626
25%       -9385.94673
50%        1840.45578
75%       24708.37428
max      415346.87583
Name: TV Change, dtype: float64

In [31]:
zero[(zero['Trade Name'] == 'dummyTrade1') & (zero['Value Date'] == '2022-09-12')]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change
4860,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,10.00000,4.27419,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.16740,0.03019,0.16845,-0.00105,5.23230,-0.00549,0.00004,-214121.70709,366.47243
4872,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,15.00000,-1.16910,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.24197,0.03019,0.24321,-0.00124,-1.40403,0.00174,0.00004,-214121.70709,366.47243
4884,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,1.50000,0.00000,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.08697,0.03019,0.08913,-0.00216,0.00000,-0.00000,0.00004,-214121.70709,366.47243
4896,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,1.00000,0.09811,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.09915,0.03019,0.10223,-0.00308,-0.01906,0.00006,0.00004,-214121.70709,366.47243
4908,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,2.00000,-0.59820,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.08022,0.03019,0.08195,-0.00173,-0.54842,0.00095,0.00004,-214121.70709,366.47243
4920,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,3.00000,-9.90792,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.07374,0.03019,0.07444,-0.00069,-9.43695,0.00652,0.00004,-214121.70709,366.47243
4932,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,4.00000,-6.70686,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.06961,0.03019,0.07001,-0.00040,-9.34633,0.00375,0.00004,-214121.70709,366.47243
4944,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,5.00000,60.16767,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.06982,0.03019,0.07053,-0.00070,64.16649,-0.04510,0.00004,-214121.70709,366.47243
4956,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,8.00000,-67.60926,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.12882,0.03019,0.12937,-0.00055,-69.43331,0.03807,0.00004,-214121.70709,366.47243
4968,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.50000,2024-03-12,10.00000,-16.18424,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.17641,0.03019,0.17803,-0.00162,-19.19440,0.03111,0.00004,-214121.70709,366.47243


In [40]:
#group by sum
totalVegaRisk = zero.groupby(['Value Date', 'Trade Name'])['Vega Risk Exposure'].sum()
# Numpy Array of Total risk shown
totalVegaRisk = pd.DataFrame(totalVegaRisk)
totalVegaRisk = totalVegaRisk.rename(columns = {'Vega Risk Exposure': 'Vega Risk Exposure (Total)'})

# Convert to a DF to merge back to orginal dataframe
zero = zero.merge(totalVegaRisk, on = ['Value Date', 'Trade Name'], how = "left")

In [42]:
zero[(zero['Trade Name'] == 'dummyTrade1') & (zero['Value Date'] == '2022-09-12')]

,Value Date,Trade Name,Trade Currency,Zero Rate Shock,Zero Rate Shock (DEC),TV,Expiry Bucket,Expiry Date,Tenor Bucket,Vega,CMS Tenor,Swap Rate,Swap Rate (Shocked),lower_bound,upper_bound,bound width,month_pay_frequency,year_maturity,Distance_lower (Shocked),Distance_upper (Shocked),Out of Bound,Swap_Bound Distance,Vols,Swap Rate Lag 1 (Real),Vols Prev,Vols Change,Vega Lag 1,Vega Risk Exposure,Swap Rate Change,TV Lag 1,TV Change,Vega Risk Exposure (Total)
4860,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,10.00000,4.27419,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.16740,0.03019,0.16845,-0.00105,5.23230,-0.00549,0.00004,-214121.70709,366.47243,-0.36306
4872,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,15.00000,-1.16910,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.24197,0.03019,0.24321,-0.00124,-1.40403,0.00174,0.00004,-214121.70709,366.47243,-0.36306
4884,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,1.50000,0.00000,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.08697,0.03019,0.08913,-0.00216,0.00000,-0.00000,0.00004,-214121.70709,366.47243,-0.36306
4896,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,1.00000,0.09811,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.09915,0.03019,0.10223,-0.00308,-0.01906,0.00006,0.00004,-214121.70709,366.47243,-0.36306
4908,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,2.00000,-0.59820,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.08022,0.03019,0.08195,-0.00173,-0.54842,0.00095,0.00004,-214121.70709,366.47243,-0.36306
4920,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,3.00000,-9.90792,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.07374,0.03019,0.07444,-0.00069,-9.43695,0.00652,0.00004,-214121.70709,366.47243,-0.36306
4932,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,4.00000,-6.70686,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.06961,0.03019,0.07001,-0.00040,-9.34633,0.00375,0.00004,-214121.70709,366.47243,-0.36306
4944,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,5.00000,60.16767,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.06982,0.03019,0.07053,-0.00070,64.16649,-0.04510,0.00004,-214121.70709,366.47243,-0.36306
4956,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.00000,2023-09-12,8.00000,-67.60926,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.12882,0.03019,0.12937,-0.00055,-69.43331,0.03807,0.00004,-214121.70709,366.47243,-0.36306
4968,2022-09-12,dummyTrade1,USD,0,0.00000,-213755.23465,1.50000,2024-03-12,10.00000,-16.18424,2,0.03023,0.03023,0.00420,0.03790,0.03370,6,5,0.02603,0.00767,False,0.00767,0.17641,0.03019,0.17803,-0.00162,-19.19440,0.03111,0.00004,-214121.70709,366.47243,-0.36306


In [46]:
zero['Vega Risk Exposure (Total)'].drop_duplicates().describe()

count    4021.00000
mean        2.18373
std       148.19848
min     -1787.41276
25%        -3.39219
50%         0.00506
75%         3.21881
max      8577.49063
Name: Vega Risk Exposure (Total), dtype: float64